In [133]:
import json
import base64


class Transaction(object):
    
    def __init__(self,sender,receiver,amount,comment):
        self.sender = sender
        self.receiver = receiver
        self.amount = amount
        self.comment = comment
        self.signature = None
        
    def __eq__(self,transaction):
        if self.sender == transaction.sender and self.receiver==transaction.receiver and self.amount==transaction.amount:
            return True
        return False
        
    @classmethod
    def new(self,sender,receiver,amount,comment):
        t1 = Transaction(sender,receiver,amount,comment)
        return t1
        
    
    def serialize(self):
        props = {}
        props["receiver"] = base64.encodebytes(self.receiver.to_string()).decode('ascii')
        props["sender"] =  base64.encodebytes(self.sender.to_string()).decode('ascii')
        props["amount"] = self.amount
        props["comment"] = self.comment
        return json.dumps(props)
    
    @classmethod
    def deserialize(self,js_string):
        obj = json.loads(js_string)
        obj['sender'] = obj['sender'].encode('ascii')
        obj['receiver'] = obj['receiver'].encode('ascii')
        result = Transaction(VerifyingKey.from_string(base64.decodebytes(obj['sender']),curve=NIST384p),VerifyingKey.from_string(base64.decodebytes(obj['receiver']),curve=NIST384p),obj['amount'],obj['comment'])
        return result 
        
    def sign(self,priv_key):
        s = self.serialize().encode()
        print(s)
        sig = priv_key.sign(s)
        print('finished signing')
        self.signature = sig
        return sig
    
    def validate(self):
        assert self.sender.verify(self.signature, self.serialize().encode()),"validation failed"
        print('finished validating')
        return
        
        
    

        



In [134]:
from ecdsa import SigningKey, VerifyingKey , NIST384p
sender_private = SigningKey.generate(NIST384p) 
sender_public = sender_private.verifying_key

In [135]:
rec_private = SigningKey.generate(NIST384p) 
rec_public = rec_private.verifying_key

In [136]:
Trans = Transaction.new(sender_public,rec_public,100,'base transaction')


In [137]:
json1 = Trans.serialize()
json1_des = Trans.deserialize(json1)
# print(Trans.receiver)
# print(json1_des.receiver)
print(Trans==json1_des)

True


In [138]:
Trans.sign(sender_private)

b'{"receiver": "pjJQ0K+5gqYH5Cym4Yo/hVXl5DEYjN3Zij2dzYlDXCvMimEpTjfaoZLFt/9L0vRTlxTyAZTq3L1s\\nGKhaPla6q5NsJ1FL9Bto4rK/1eOa9cbWbKOHQe4Ggh344N4mr+6P\\n", "sender": "kZ33XvQ7tQIHO72BweC1axKizAaHpgCslBjFatTzMHoxJffPDJrg6jB74BNrr+GnZGund28enUBd\\n9xhn4TLfDxuPk5IPSJfyeX57DHTloYJMeNrB+fqHHYCrEtvyZl60\\n", "amount": 100, "comment": "base transaction"}'
finished signing


b"\x15Dz\x97\x83\x0bM'\xb8\x8d7\x8c\xff\xaf\x8cG\xb1.\x90\x075T@\xf5\xc0\x813\xe8\xc0m\xda\xe5xSgY\n\x05\x85F[\xf2B\xdd\x17n\xe5\xe7\x92\x13\x9c\xca\xf9\x8f\xabt:\xcbg\x9e\xc8\xea\x17N\xfe\xed\x9f7<\x99\xa5\xd8\x90\x07[\xd1\x89@\x92/\x86\x1a\x95\xf77&cR9\xc2fZ\x070\xdc\xa3"

In [139]:
Trans.validate()

finished validating
